<a href="https://colab.research.google.com/github/jpgerber/Recommender-for-movie-snobs/blob/master/0_Movie_Snob_Data_Clean_%26_Wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook preps the datafiles for the ML.
It first makes a file that adds canonical status to the ratings file.
<p>Then, we make the indicators of snobbery.
<p>Both of these are saved to my Google Drive.


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import zipfile, io

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#### First stage is to add the canonical indicator to the movie titles file

In [2]:
# Make the canonical list
# Importing the 1,001 list and converting it to a list
snob_url = 'https://1001films.fandom.com/wiki/The_List'
snob_text= requests.get(snob_url)
soup = BeautifulSoup(snob_text.content, 'html.parser')
basic_list = (soup.body.find_all('b'))
thousand_list = [item.text for item in basic_list]
thousandone_movies = pd.DataFrame(thousand_list, columns = ['title']).drop(0) # Convert to df


In [3]:
# Get the MovieLens dataset
# Importing the ratings data
list_of_urls = ['http://files.grouplens.org/datasets/movielens/ml-latest.zip'] # I originally checked several files
for url in list_of_urls:
    ratings_small_file = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(ratings_small_file.content))
    z.extractall()

gl_movies = pd.read_csv('ml-latest/movies.csv', sep = ',', header = 0) # Make the df


##### There will be some different types of cleaning.
First, extract the year of release from the string titles to enable easier title matching

In [4]:
# Create columns of movie years in each database
# Make sure the titles don't have trailing spaces
thousandone_movies['title'] = thousandone_movies['title'].str.rstrip()
gl_movies['title'] = gl_movies['title'].str.rstrip()
# Then take the slices (the years are in parantheses at the end of the title)
thousandone_movies['year'] = [title[slice(-5,-1)] for title in thousandone_movies['title']]
gl_movies['year'] = [title[slice(-5,-1)] for title in gl_movies['title']]

# Then convert these strings to numbers (there is one title missing a year!)
# Define a conversion function
def ConvertYear(value):
    '''This function converts integer strings to integers and non-integer strings to zero'''
    try:
        return int(value)
    except: 
        return 0
# Then apply it to the columns for both thousandone_movies and ed_choices
thousandone_movies['year'] = thousandone_movies['year'].apply(lambda year: ConvertYear(
    year))
gl_movies['year'] = gl_movies['year'].apply(lambda year: ConvertYear(year))


Then make a custom function to select a three-year window to more efficiently do string matching.
Then run that matching

In [5]:
!pip install fuzzywuzzy 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# Specify the matching function (we only need one of the outputs)
def Matcher(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices)
    return title_match
# And here's a function for using the tokenizer
def Matcher_token(title, choices):
    title_match, percent_match, match3 = process.extractOne(title, choices, 
                                                            scorer=fuzz.token_sort_ratio)
    return title_match

#Define a filter to return targets for +/-1 year only
def YearFilter (year):
    years = [year-1, year, year+1]
    return gl_movies[gl_movies.year.isin(years)].title

# Running the tokenizer over the filtered target set
for index, row in thousandone_movies.iterrows():
    # call the filter
    targets = YearFilter(row.year)
    # update the new cell work out the matcher
    thousandone_movies.loc[index,'return_match'] = Matcher_token(row.title, targets)


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



The following movies were misidentified and so needed re-coding
Intolerance (1916) - 7243
Broken Blossoms (1919) - 6988
Häxan (1923) - 25744
Sunrise (1927) - 8125
The Unknown (1927) - 25762
A Throw of Dice (Prapancha Pash) (1929) - NONE
Tabu (1931) - 5599
The Vampire (Vampyr) (1932) - 25793
Scarface: The Shame of a Nation (1932) - 25788
Midnight Song (Ye Ban Ge Sheng) (1937) - None
Henry V (1944) - 25901
The Battle of San Pietro (1945) - 80104
Gun Crazy (1949) - 8751
Sunset Blvd. (1950) - 922
Europa '51 (1952) - 25966
Tokyo Story (1953) - 6643
The Wanton Countess (Senso) (1954) - 69911
The Sins of Lola Montes (Lola Montès) (1955) - 8143
Pather Panchali (1955) - 668
Ordet (1955) - 6981
Hill 24 Doesn't Answer (1955) - NONE
Dracula (1958) - 5649
Dog Star Man - 137579	137581	137583	137585	137587
Blonde Cobra (1963) - None
Playtime (1967) - 26171
Week End (1967) - 7749
Viy (1967) - 97065
Andrei Rublev (Andrei Rublyov) (1966) - 26150
A Touch of Zen (Hsia Nu) (1969) - 32511
M*A*S*H (1970) - 5060
The Sorrow and the Pity (La Chagrin et la Pitié) (1971) - 32853
Ceddo (1977) - 71973
Up in Smoke (1978) - 1194
Raiders of the Lost Ark (1981) - 1198
Yol (1982) - 6151
Koyaanisqatsi (1983) - 1289
The Naked Gun (1988) - 3868
Henry: Portrait of a Serial Killer (1990) - 2159
The Actress (Yuen Ling-Yuk) (1992) - 114394
Hana-Bi (1997) - 1809
Buffalo '66 (1998) - 1916
Tetsuo (1989) - 4552
A One and a Two (Yi Yi) (2000) - 4334
Y Tu Mama Tambien (2001) - 5225
Oldboy (2003) - 107314
Paranormal Activity (2007) - 71379
Precious: Based on the Novel "Push" by Sapphire (2009) - 72395
The Favourite (2018) - 183837
Vice (2018) - 127323

Join the canonical indicators list to the titles file.

In [6]:
# Add the indicator variable to the canonical list.
thousandone_movies['canonical'] = 1
#print(thousandone_movies.head())

# Add the canonical indicator to the movie file, drop the irrelevant columns 
#and fill the missing values with zeroes
gl_movies = pd.merge(gl_movies, thousandone_movies, left_on='title', right_on='return_match', how='outer', 
         suffixes=('', '_canon')).drop(['year_canon', 
                                        'return_match', 'title_canon'], axis=1).fillna({'canonical':0})

print(gl_movies.head())

   movieId                               title  ...  year  canonical
0        1                    Toy Story (1995)  ...  1995        1.0
1        2                      Jumanji (1995)  ...  1995        0.0
2        3             Grumpier Old Men (1995)  ...  1995        0.0
3        4            Waiting to Exhale (1995)  ...  1995        0.0
4        5  Father of the Bride Part II (1995)  ...  1995        0.0

[5 rows x 5 columns]


In [7]:
# Now add the mismatched ones

handcode_ids = [7243, 6988, 25744, 8125, 25762, 5599, 25793, 25788, 25901, 80104,
                         8751, 922, 25966, 6643, 69911, 8143, 668, 6981, 5649, 137579,
                         137581, 137583, 137585, 137587, 26171, 7749, 97065, 26150, 32511,
                         5060, 32853, 71973, 1194, 1198, 6151, 1289, 3868, 2159, 114394,
                         1809, 1916, 4552, 4334, 5225, 107314, 71379, 72395, 183837, 127323]

def handcode_row(row):
    if row['movieId'] in handcode_ids or row['canonical'] == 1:
      return 1
    else:
      return 0

gl_movies['canonical'] = gl_movies.apply(lambda row : handcode_row(row), axis=1) 



In [8]:
gl_movies.head()

,movieId,title,genres,year,canonical
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,1
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,0
4,5,Father of the Bride Part II (1995),Comedy,1995,0


In [9]:
# Quickly check that there are 1,200 canonical movies
gl_movies.canonical.value_counts()

0    56829
1     1270
Name: canonical, dtype: int64

### Now join the canonical indicator to the ratings file 
(movie and ratings file have the same movieId)

In [10]:
# Load the ratings file and convert the rating date
import datetime
ratings = pd.read_csv('ml-latest/ratings.csv', sep = ',', header = 0)
ratings['rating_date'] = ratings['timestamp'].apply(lambda x: datetime.date.fromtimestamp(x))

print(ratings.head())


   userId  movieId  rating   timestamp rating_date
0       1      307     3.5  1256677221  2009-10-27
1       1      481     3.5  1256677456  2009-10-27
2       1     1091     1.5  1256677471  2009-10-27
3       1     1257     4.5  1256677460  2009-10-27
4       1     1449     4.5  1256677264  2009-10-27


In [11]:
# Here's the main merge
rating2 = pd.merge(ratings, gl_movies.drop_duplicates(subset=['movieId']), how='left', on='movieId').drop(
    ['title','genres','timestamp'], axis=1)  # we also dropped the long string columns and some others
rating2.head()

,userId,movieId,rating,rating_date,canonical
0,1,307,3.5,2009-10-27,1
1,1,481,3.5,2009-10-27,0
2,1,1091,1.5,2009-10-27,0
3,1,1257,4.5,2009-10-27,0
4,1,1449,4.5,2009-10-27,0


In [12]:

# Removing the one suspicious user
rating2 = rating2[rating2.userId != 123100]
rating2.groupby('userId').agg({'rating': 'count'}).sort_values(by='rating', ascending=False)


,rating
userId,
117490,9279
134596,8381
212343,7884
242683,7515
111908,6645
...,...
188125,1
117282,1
127062,1


## Saving the initial ratings file to my Google Drive

In [13]:
rating2.to_csv('ratings.zip', compression={'archive_name':'rating2.csv','method':'zip'}, index=False)
# File save
! cp ratings.zip '/content/gdrive/My Drive/'
! ls '/content/gdrive/My Drive/'


'Capstone #2: Final report.gdoc'
'Copy of TEMPLATE_Track companies and contacts.gsheet'
 Data
 foo.txt
'Getting started.pdf'
"Imbalanced classes in Naive Bayes' classifiers.gdoc"
'Notes on three informational interviews.gdoc'
 ratings.zip
'Statistical techniques database.gsheet'


### Reading that file back

In [14]:
ratings_df = pd.read_csv('/content/gdrive/My Drive/ratings.zip')
ratings_df.head()

,userId,movieId,rating,rating_date,canonical
0,1,307,3.5,2009-10-27,1
1,1,481,3.5,2009-10-27,0
2,1,1091,1.5,2009-10-27,0
3,1,1257,4.5,2009-10-27,0
4,1,1449,4.5,2009-10-27,0


### Now to make the file with all the snobbery indices